# Tracking notebook
This notebook performs the tracking of the tip of the club from an experiment video. It uses openCV template matching. The following cell reads the video and return the tracking view as well as the velocity of the club at each frame.

In [8]:
import cv2
import numpy as np

# Get the video file and read it
video = cv2.VideoCapture("experimentSet2.mp4")
ret, frame = video.read()
resize_factor = 2
frame_height, frame_width = frame.shape[:2]

tail_pixels = 310
length_tail = 310/resize_factor
real_length_tail = 1

template = []
res = []
for i in range(5):
    template.append(cv2.imread("templates/temp"+str(i+1)+".png"))
    template[i] = cv2.resize(template[i], [150//resize_factor, 150//resize_factor])
    template[i] = cv2.cvtColor(template[i], cv2.COLOR_BGR2GRAY)
    template[i] = cv2.GaussianBlur(template[i], (3, 3), 0)

h, w = template[0].shape[:2]
fps_init = 50

def frame_transform(ret, frame):
    # transform the frame to optimize performance
    frame = cv2.resize(frame, [frame_width//resize_factor, frame_height//resize_factor])
    frame = frame[0:int(frame_height*0.57/resize_factor), int(frame_width*0.2/resize_factor):int(frame_width*0.8/resize_factor)]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame, (3, 3), 0)
    frame = cv2.copyMakeBorder(frame, h, h, w, w, borderType=cv2.BORDER_CONSTANT)
    return ret, frame

# Resize the video for a more convinient view
ret, frame = frame_transform(ret, frame)
# Initialize video writer to save the results
output = cv2.VideoWriter('results.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 50, (frame.shape[1], frame.shape[0]), True)

if not ret:
    print('cannot read the video')
# Start tracking
top_left = (0,0)
bottom_right= (0,0)
frame_count = 0
max_loc =(0,0)
speedF = []
area_top_left = (0,0)
area_bottom_right = (frame_width, frame_height)
while True:
    res = []

    ret, frame = video.read()
    if not ret:
        print('something went wrong')
        break
    ret, frame = frame_transform(ret, frame)
    prev_max_loc = max_loc

    frame_count +=1
    timer = cv2.getTickCount()
    max_val = 0

    if frame_count > 1:
        area_top_left = (int(top_left[0]-w), int(top_left[1]-h))
        area_bottom_right = (area_top_left[0] +3*w, area_top_left[1]+3*h)
    crop = frame[area_top_left[1]:area_bottom_right[1], area_top_left[0]:area_bottom_right[0]]

    for i in range(5):
        res.append(cv2.matchTemplate(crop,template[i], cv2.TM_CCOEFF_NORMED))
        if cv2.minMaxLoc(res[i])[1] > max_val:
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res[i])

    top_left = (max_loc[0]+area_top_left[0], max_loc[1]+area_top_left[1])
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    cv2.rectangle(frame,top_left, bottom_right, (255, 0 ,0), 2)
    cv2.rectangle(frame,area_top_left, area_bottom_right, (0, 255 ,0), 2)
    cv2.putText(frame, "Frame : " + str(int(frame_count)), (100,50), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.imshow("Tracking", frame)

    output.write(frame)
 
    vec = (prev_max_loc[0] - max_loc[0], prev_max_loc[1] - max_loc[1])
    speedF.append(np.linalg.norm(vec)*fps_init*real_length_tail/length_tail)

    k = cv2.waitKey(1) & 0xff
    if k == 27 : break
    
video.release()
output.release()
cv2.destroyAllWindows()

something went wrong


The next cell slice the experimentations in the video to extract the speed of each experiment. This is an example, it needs to be adapted depending on the video used.

In [95]:
import matplotlib.pyplot as plt

start = 525
end = 17700
n_experiment = 55
length = int((end-start)/n_experiment)
length2 = int((end2-start2)/9)
experiment = []
for i in range(n_experiment):
    experiment.append(speed2[start + i*length:start + (i+1)*length])

blow_time = []; body_rot = []; body_rot_offset = []; body_vel = []; tail_vel = []; other = []; factor = []

for i in range(54):
    if i <9:
       factor.append(exp2[i])
    if i<9:
        blow_time.append(experiment[i])

    elif i <9+7:
        body_rot.append(experiment[i])

    elif i<9+7+9:
        body_rot_offset.append(experiment[i])

    elif i<9+7+9+10:
        body_vel.append(experiment[i])
        
    elif i<9+7+9+10+10:
        tail_vel.append(experiment[i])
    else:
        other.append(experiment[i])
